In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# Define dataset path
dataset_path = "/kaggle/input/face-detection-by-kazmir/Face detection"

# Load dataset with data augmentation
batch_size = 16
img_size = (224, 224)

# Load the dataset before augmentation to retrieve class names
train_ds, val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

# Check the number of classes by inspecting the class names
class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Classes found: {class_names}")

# Now apply data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.2),
    layers.GaussianNoise(0.1),  # Adding Gaussian noise
])


# Apply the augmentation to the dataset
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

# Use the MobileNetV2 pre-trained model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model to use it as a feature extractor
base_model.trainable = False

# Add classification layers on top with dropout to prevent overfitting
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling layer
x = Dense(128, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Add dropout layer with 50% dropout rate
predictions = Dense(num_classes, activation='softmax')(x)  # Number of classes

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=5,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored metric
)


# Train the model
epochs = 20  # Increased the number of epochs

# Fit the model with the early stopping callback
history = model.fit(
    train_ds,
    validation_data=val_ds,  
    epochs=20,               
    callbacks=[early_stopping]
)

# Save the model
model.save('/kaggle/working/face_detection_model_with_augmentation.h5')




Found 25 files belonging to 5 classes.
Using 20 files for training.
Using 5 files for validation.
Classes found: ['Kazmir', 'Mudassir', 'ashik', 'riyan', 'sayed']
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.3167 - loss: 1.7688 - val_accuracy: 0.4000 - val_loss: 1.5452
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 230ms/step - accuracy: 0.3917 - loss: 1.4493 - val_accuracy: 0.4000 - val_loss: 1.2781
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step - accuracy: 0.6083 - loss: 0.9477 - val_accuracy: 0.6000 - val_loss: 1.1494
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step - accuracy: 0.5875 - loss: 1.0833 - val_accuracy: 0.6000 - val_loss: 1.0132
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 245ms/step - accuracy: 0.5333 - loss: 1.0822 - val_accuracy: 0.6000 - val_loss: 0.9493
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 210ms/step - accuracy: 0.7167 - loss: 0.9696 - val_accuracy: 0.6000 - val_loss: 0.9846
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step - accuracy: 0.5000 - loss: 0.9754 